In [8]:
# MNIST save model p3 ( working with ImageFolder)
import os
import torch
import torch.utils.data as data
import json
from PIL import Image
import torchvision.transforms.v2 as tfs
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.datasets import ImageFolder


class RavelTransform(nn.Module):
    def forward(self,item):
        return item.ravel()


class DigitNN(nn.Module):
    def __init__(self, input_dim, num_hid, out_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim,num_hid)
        self.layer2 = nn.Linear(num_hid, out_dim)

    def forward(self,x):
        x = self.layer1(x)
        x = nn.functional.relu(x)
        x = self.layer2(x)
        return x

model = DigitNN(28 * 28, 32, 10)


# Setting trnsform
transforms = tfs.Compose([tfs.ToImage(),
                          tfs.Grayscale(), 
                          tfs.ToDtype(torch.float32, scale=True), # converting to float
                          RavelTransform() # or tfs.Lambda(lambda _im: _im.ravel()),
                          ])
d_dataset = ImageFolder("datasets/MNIST/train", transform=transforms)
train_data = data.DataLoader(d_dataset, batch_size=32, shuffle=True)

it = iter(train_data)
x, y  = next(it)


optimizer = optim.Adam(params=model.parameters(), lr=0.01)
loss_function = nn.CrossEntropyLoss()

ep = 2
model.train()

for _e in range(ep):
    train_tqdm = tqdm(train_data, leave=True)

    for x_train_img, y_target in train_tqdm:
        predict = model(x_train_img)
        loss = loss_function(predict, y_target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

st = model.state_dict() # Веса и смещения для слоя
torch.save(st,"models/mnist_model.tar" )

# Test
d_test = ImageFolder("datasets/MNIST/test", transform=transforms)
test_data = data.DataLoader(d_test , batch_size=500, shuffle=False)
Q = 0

model.eval()

test_tqdm = tqdm(test_data, leave=True)
for x_test, y_test in test_tqdm:    
    p = model(x_test) # 500, 10
    p = torch.argmax(p, dim=1) # 500,1
    Q += torch.sum(p == y_test).item()

Q /= len(d_test)
print(Q)



100%|██████████| 20/20 [00:02<00:00,  7.07it/s]

0.9478


In [9]:
# Test
d_test = ImageFolder("datasets/MNIST/test", transform=transforms)
test_data = data.DataLoader(d_test , batch_size=500, shuffle=False)
Q = 0

st_dict = torch.load("models/mnist_model.tar", weights_only=True)
model.load_state_dict(st_dict)
model.eval()

test_tqdm = tqdm(test_data, leave=True)
for x_test, y_test in test_tqdm:    
    p = model(x_test) # 500, 10
    p = torch.argmax(p, dim=1) # 500,1
    Q += torch.sum(p == y_test).item()

Q /= len(d_test)
print(Q)


100%|██████████| 20/20 [00:02<00:00,  6.90it/s]

0.9478


In [ ]:
# 3
import torch
from collections import OrderedDict

# эти тензоры в программе не менять
layer1 = torch.rand(64, 32)
bias1 = torch.rand(32)
layer2 = torch.rand(32, 10)
bias2 = torch.rand(10)

# здесь продолжайте программу
data_w = OrderedDict()
data_w["layer1"] = layer1
data_w["bias1"] = bias1
data_w["layer2"] = layer2
data_w["bias2"] = bias2

torch.save(data_w, 'data_w.tar')

data_w2 = torch.load('data_w.tar')

In [3]:
# 4
import torch
import torch.nn as nn

# здесь продолжайте программу
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(13,32)
        self.layer2 = nn.Linear(32,16)
        self.layer3 = nn.Linear(16,3)
    def forward(self, x):
        x = self.layer1(x).relu()
        x = self.layer2(x).relu()
        x = self.layer3(x)
        return x

model = MyModel()
st = model.state_dict()
torch.save(st, 'func_nn.tar')


In [ ]:
# 5
import torch
import torch.nn as nn

# здесь объявляйте класс модели
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(48,32, bias=False)
        self.layer2 = nn.Linear(32,16)
        self.out = nn.Linear(16,10)
    def forward(self, x):
        x = self.layer1(x).relu()
        x = self.layer2(x).relu()
        x = self.out(x)
        return x


x = torch.ones(48) # тензор в программе не менять

# здесь продолжайте программу
model = MyModel()
st_dict = torch.load('toy_nn.tar', weights_only=True)
model.load_state_dict(st_dict)
model.eval()
predict = model(x)

In [ ]:
# 6
import torch
import torch.nn as nn
import torch.optim as optim

# здесь продолжайте программу
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(25, 16, bias=False)
        self.layer2 = nn.Linear(16, 8)
        self.out = nn.Linear(8, 5, bias=False)
    def forward(self, x):
        x = self.layer1(x).tanh()
        x = self.layer2(x).tanh()
        x = self.out(x)
        return x
    
model = MyModel()    
opt = optim.Adam(params=model.parameters(), lr=0.02)
loss_func = nn.CrossEntropyLoss()

data_state_dict = {
    'loss': loss_func.state_dict(),
    'opt': opt.state_dict(),
    'model': model.state_dict()
}
torch.save(data_state_dict, 'my_data_state.tar')

In [ ]:
# 7
import torch
import torch.nn as nn
import torch.optim as optim


# здесь продолжайте программу
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(10, 8)
        self.layer2 = nn.Linear(8, 4)
        self.out = nn.Linear(4, 6)
    def forward(self, x):
        x = self.layer1(x).sigmoid()
        x = self.layer2(x).sigmoid()
        x = self.out(x)
        return x
    
model = MyModel()
opt = optim.RMSprop(params=model.parameters(), lr = 0.05)
st = torch.load('nn_data_state.tar', weights_only=True)

model.load_state_dict(st["model"])
opt.load_state_dict(st["opt"])

In [ ]:
# 9 
import torch

# w
x1, y1 = 0, 3
x2, y2 = -2, 0

delta_x = x2 - x1
delta_y = y2 - y1

# Ax + By + C = 0
A = delta_y
B = -delta_x
C = delta_x * y1 - delta_y * x1

w = torch.tensor([C, A, B],dtype=torch.float32)